# テーマ2

# NG_特定のカラムだけにする

In [ ]:
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# 例としてデータロードとモデル訓練を行います
uploaded = files.upload()

# CSVファイルを読み込む
df = pd.read_csv('train.csv')

# ヘッダー行から列名を動的に取得する
column_names_to_extract = ['1', '2', '3', '4', '5', '6', '15', '16', '17', '18', '23', '24']  # 必要な列名をリストで指定

# 動的に取得した列名を使ってデータを抽出する
extracted_columns = df[column_names_to_extract]

# 抽出したデータを新しいCSVファイルとして保存する
extracted_columns.to_csv('extracted_data.csv', index=False)

# ファイルが保存されたことを確認するためにメッセージを表示
print("Extracted data has been saved to extracted_data.csv")

Saving train.csv to train (5).csv


KeyError: "None of [Index(['1', '2', '3', '4', '5', '6', '15', '16', '17', '18', '23', '24'], dtype='object')] are in the [columns]"

# データ補完(湿度の補完) 降水量*50で100以内

In [7]:
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# 例としてデータロードとモデル訓練を行います
uploaded = files.upload()

# Read the CSV file
df = pd.read_csv('extracted_train.csv')
# print(df)

# Handle rows where the 5th column is empty
df['Humidity3pm'] = df.apply(
    # lambda row: (int(row.iloc[3]) * 50) if pd.isnull(row.iloc[4]) else row.iloc[4],
    lambda row: min(int(row.iloc[3]) * 50, 100) if pd.isnull(row.iloc[4]) else row.iloc[4],
    axis=1
)
# Filter rows where the 5th column was filled (to avoid NaN values)
df = df[~pd.isnull(df['Humidity3pm'])]
# print(df)

# Select the columns you want to save
columns_to_save = ['id', 'Date', 'Location', 'Rainfall', 'Humidity3pm', 'RainToday', 'RainTomorrow']  # Replace with actual column names

# Save the processed data to a new CSV file
df[columns_to_save].to_csv('hokann_extracted_train.csv', index=False)

Saving extracted_train.csv to extracted_train.csv


# テーマ1からパクり

# 文字列変換

In [ ]:
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# 例としてデータロードとモデル訓練を行います
uploaded = files.upload()

# TSVファイルをロード
df = pd.read_csv('train.tsv', sep='\t', dtype=str)

# 変換ルールを定義
conversion_map = {
    'vhigh': 1,
    'high': 2,
    'med': 3,
    'low': 4,
    'small':5,
    'big':6,
}

# 3カラム目（通常はインデックス2）を条件に基づいて変換
column_index = 2  # ここを変えれば別のカラムを指定できます

def convert_value(value):
    try:
        # もともと数字の場合はそのまま返す
        return int(value)
    except ValueError:
        # 変換マップで指定されている値ならマップの値を、それ以外なら5を返す
        return conversion_map.get(value, 7)

# 3カラム目以降の全てを変換する
for column in df.columns[2:]:
    df[column] = df[column].apply(convert_value)

# 変換後のデータフレームを表示
print(df)

# 必要に応じて、新しいTSVファイルに保存
df.to_csv('int_train.tsv', sep='\t', index=False)
files.download('int_train.tsv')

# 決定木モデル作成

In [ ]:
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import files

# 訓練用データを読み込む
uploaded = files.upload()
train_file = 'int_train8.tsv'
train_data = pd.read_csv(train_file, sep='\t', header=0)

# 説明変数と目的変数を分ける
X_train = train_data.iloc[1:, 2:8]  # 3,4,5,6,7,8カラム目が説明変数
y_train = train_data.iloc[1:, 1]    # 2カラム目が目的変数

# 各カラムのデータ型を出力
print("=== Initial Data Types (train8) ===")
print(X_train)
print("------")
print(X_train.dtypes)

# モデルを作成し訓練する
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

# モデルを保存する
model_filename = 'int_decision_tree_model.joblib'
joblib.dump(model, model_filename)
# データの保存
files.download('int_decision_tree_model.joblib')

### 以下デバッグ用
# 評価用データを読み込む
#test_file = 'int_train2.tsv'
#test_data = pd.read_csv(test_file, sep='\t', header=0)

# 2行目以降の3,4,5,6,7,8カラム目を説明変数として取得（ヘッダを含まない）
#test_data = test_data.iloc[:, 2:8]  # 2行目以降を取得

# 各カラムのデータ型を出力
#print("=== Initial Data Types (train2) ===")
#print(test_data)
#print("------")
#print(test_data.dtypes)

# データが数値型であるか確認し、必要に応じて変換する
#test = test.apply(pd.to_numeric, errors='coerce')

# 新しいデータで目的変数を予測する
#predictions = model.predict(test_data)

# 予測結果を出力する
#output = pd.DataFrame(predictions, columns=['Predicted'])
#output.to_csv('predictions.csv', index=False)
# データの保存
#files.download('predictions.csv')


Saving int_train2.tsv to int_train2 (4).tsv
Saving int_train8.tsv to int_train8 (4).tsv
=== Initial Data Types (train8) ===
     buying  maint  doors  persons  lug_boot  safety
1         3      2      3        4         3       2
2         2      2      7        7         5       4
3         2      3      3        2         6       4
4         2      4      7        4         5       4
5         4      2      2        2         3       3
..      ...    ...    ...      ...       ...     ...
686       2      2      2        4         5       2
687       2      4      2        7         3       3
688       2      2      7        4         5       3
689       2      2      3        4         5       2
690       1      1      3        4         5       2

[690 rows x 6 columns]
------
buying      int64
maint       int64
doors       int64
persons     int64
lug_boot    int64
safety      int64
dtype: object


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# モデル評価

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み
train_file = 'int_train8.tsv'
val_file = 'int_train2.tsv'

# データの読み込み
train_data = pd.read_csv(train_file, sep='\t', header=None)
val_data = pd.read_csv(val_file, sep='\t', header=None)

# 1行目が不要なら削除（場合によって削除が必要）
train_data = train_data.drop(index=0).reset_index(drop=True)
val_data = val_data.drop(index=0).reset_index(drop=True)

# 説明変数と目的変数の分割
X_train = train_data.drop(columns=1)
y_train = train_data[1]

X_val = val_data.drop(columns=1)
y_val = val_data[1]

# Categorical feature encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

# 対応する列を合わせる（片方にしかない列をカバー）
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

# 決定木モデルのインスタンス作成と訓練
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# 検証データでの予測と評価
predictions = clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 必要に応じて、分類レポートや混同行列の出力も可能
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_val, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, predictions))

Saving int_train2.tsv to int_train2 (8).tsv
Saving int_train8.tsv to int_train8 (8).tsv
Accuracy: 94.80%

Classification Report:
              precision    recall  f1-score   support

         acc       0.89      0.93      0.91        43
        good       0.75      0.43      0.55         7
       unacc       0.98      0.99      0.99       119
       vgood       0.75      0.75      0.75         4

    accuracy                           0.95       173
   macro avg       0.84      0.78      0.80       173
weighted avg       0.95      0.95      0.94       173


Confusion Matrix:
[[ 40   1   2   0]
 [  3   3   0   1]
 [  1   0 118   0]
 [  1   0   0   3]]


# 実践

In [ ]:
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import files

# 保存されたモデルを読み込む
uploaded = files.upload()
model_filename = 'int_decision_tree_model.joblib'
model = joblib.load(model_filename)

# 評価用データを読み込む
test_file = 'int_test.tsv'
test_data = pd.read_csv(test_file, sep='\t', header=0)
### デバッグ
# print("---test_data---")
# print(test_data)

# 2行目から説明変数を取得（2,3,4,5,6,7カラム目を使用）
X_test = test_data.iloc[0:, 1:7]  # 2行目以降の2-7カラム目を使用

### デバッグ
# print("---X_test---")
# print(X_test)
# データが数値型であるか確認、必要に応じて変換する
# X_test = X_test.apply(pd.to_numeric, errors='coerce')

# 予測を行う
predictions = model.predict(X_test)

# 出力結果のDataFrameを作成
output = pd.DataFrame({
    'Original_Value': test_data.iloc[0:, 0].values,  # 1カラム目のデータ
    'Predicted': predictions  # モデルの予測値
})

# CSVファイルに出力
# CSVに保存（ヘッダー無し）
result_filename = 'int_tree_result.csv'
output.to_csv(result_filename, index=False, header=False)
print(f"Predictions saved to {result_filename}")
files.download('int_tree_result.csv')

Saving int_decision_tree_model.joblib to int_decision_tree_model (12).joblib
Saving int_test.tsv to int_test (10).tsv
Predictions saved to int_tree_result.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>